In [ ]:
# Standard library imports
import os
import re
import csv
import sys
import html
from datetime import datetime
from collections import defaultdict

# Third-party library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd
import psutil
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Local application/library specific imports
import utils
from utils import *



from unidecode import unidecode
import importlib

## 1. Filtering french brands

Here, we'll aim to select only the french brands. 
- One approach is to use language detection. Via the gcd3 library (?)
- Another approach could be to look at location


Plan:
- Load the df of the brands and their bios. 
- Filter
- Now, filter the reduced follower df based on these final french brands

- are all users now french ? (look at combination of language and location - potentially drop those that does not have any usable indicator)


- Might need to manually inspect these later on

### Preprocessing data files

In [ ]:
# #Load and rename markersIDs in markers_bios

# load_path = '/home/livtollanes/SocialMarkers'
# save_path = '/home/livtollanes/NewData'
# file = 'markers_bios_2023-05-19.csv'


# req_cols = ['twitter_name', 'id', 'screen_name', 'description', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists','timestamp_utc']
# dtypes = {'twitter_name':'object', 
#           'id': 'float64',
#           'screen_name': 'object', 
#           'description': 'object',
#           'location': 'object',
#           'tweets': 'float64',
#           'followers': 'float64',
#           'friends': 'float64',
#           'likes': 'float64',
#           'lists': 'float64',
#           'timestamp_utc': 'float64'}

# new_column_names = {'id': 'marker_id'}

# markers_bios = utils.load_and_rename(load_path, save_path, file, req_cols, dtypes, new_column_names)


In [ ]:
# #Load and rename markers followers
# load_path = '/home/livtollanes/SocialMarkers'
# save_path = '/home/livtollanes/NewData'
# file = 'markers_followers_2023-05-19.csv'

# req_cols = ['id', 'follower_id']
# dtypes = {'id': 'float64',
#           'follower_id': 'float64'}

# new_column_names = {'id': 'marker_id'}

# markers_followers = utils.load_and_rename(load_path, save_path, file, req_cols, dtypes, new_column_names)

In [ ]:
# #Load and rename markers followers bios
# load_path = '/home/livtollanes/SocialMarkers'
# save_path = '/home/livtollanes/NewData'
# file = 'markers_followers_bios_2023-05-19.csv'


# req_cols = ['twitter_id', 'id', 'screen_name', 'description', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists','timestamp_utc']

# dtypes = {
#     'twitter_id': 'float64',
#     'id': 'float64',
#     'screen_name': 'object',
#     'description': 'object',
#     'location': 'object',
#     'tweets': 'float64',
#     'followers': 'float64',
#     'friends': 'float64',
#     'witheld_in_countries': 'float64'
# }

# new_column_names = {'twitter_id': 'follower_id'}

# follower_bios = utils.load_and_rename(load_path, save_path, file, req_cols, dtypes, new_column_names)

Simply load the files, if renaming already has been done

In [ ]:
# #Load marker bios
# path = '/home/livtollanes/NewData'
# file = 'markers_bios_2023-05-19.csv'
# req_cols = ['twitter_name', 'marker_id', 'screen_name', 'description', 'location', 'tweets', 'followers', 'friends', 'likes', 'lists','timestamp_utc']

# dtypes = {'twitter_name':'object', 
#           'marker_id': 'float64',
#           'screen_name': 'object', 
#           'description': 'object',
#           'location': 'object',
#           'tweets': 'float64',
#           'followers': 'float64',
#           'friends': 'float64',
#           'likes': 'float64',
#           'lists': 'float64',
#           'timestamp_utc': 'float64'}

# markers_bios = utils.fileloader(path, file, req_cols, dtypes)

In [ ]:
#Load marker followers
load_path = '/home/livtollanes/NewData'
file = 'markers_followers_2023-05-19.csv'

req_cols = ['marker_id', 'follower_id']
dtypes = {'marker_id': 'float64',
          'follower_id': 'float64'}


markers_followers = utils.fileloader(load_path, file, req_cols, dtypes)

In [ ]:
# Load marker follower bios

#Didn't work but the df is currently loaded in the notebook
follower_bios.head()



In [ ]:
#reload(utils)
importlib.reload(utils)

#How is my data delimited?
path = '/home/livtollanes/NewData'
file1 = 'markers_bios_2023-05-19.csv'
file2 = 'markers_followers_2023-05-19.csv'
file3 = 'markers_followers_bios_2023-05-19.csv'


utils.print_lines(path, file1, 0,1)
utils.print_lines(path, file2, 0,1)
utils.print_lines(path, file3, 0,2)

#The data is comma delimited

### Now filtering on language

In [ ]:
# Remove emojis, weird font, and detect language in descriptions

# Process the descriptions in the DataFrame
markers_bios = utils.process_description(markers_bios)

# Split the DataFrame by language
markers_bios_french, markers_bios_other = utils.split_by_language(markers_bios, 'fr')

# Finally, print information about the resulting DataFrames
utils.print_df_info(markers_bios_french, 'markers_bios')
utils.print_df_info(markers_bios_other, 'markers_bios_other')

Manual inspection - did we miss any brands that are french?
- This part was done by inspecting the data frame and looking up brands

The brands that were incorrectly detected as non french:

- 21: Lafuma_France
- 30: CarrefourFrance
- 36: CasinoEnseigne
- 37: Supermarche_G20
- 38: VogueFrance
- 39: FigaroMagazine
- 41: LeMediaTv
- 48: BFMTV
- 92: TeleLoisirs
- 94: ParisMatch
- 96: Telerama
- 101: EntMagazine
- 106: OnzeMondial
- 113: GQ_France
- 119: LEXPRESS
- 121: courrrierinter
- 124: RCLens
- 128: OL
- 129: ognice
- 131: StadeDeReims
- 133: MontpellierHSC
- 135: RCSA
- 171: HECParis
- 178: SciencesPo
- 181: Univbordeaux
- 182: UnivLyon1
- 199: UniversiteCergy
- 202: centralesupelec
- 208: ENSAEparis
- 215: esdes_BS
- 221: LaCoudouliere
- 226: LyceeProTissie
- 227: PSVLaTournelle
- 230: Decathlon
- 231: Darty_Officiel
- 233: Fnac
- 236: AmazonFrance

In [ ]:
# Add extra brands that were incorrectly defined as non-french:
indices_to_change = [21, 30, 36, 37, 38, 39, 41, 48, 92, 94, 96, 101, 106, 113, 119, 121, 124, 128, 129, 131, 133, 135, 171, 178, 181, 182, 199, 202, 208, 215, 221, 226, 227, 230, 231, 233, 236]

#Add and save file
path_tosave = '/home/livtollanes/NewData/workdata'
utils.add_extrabrands(indices_to_change, markers_bios_other, markers_bios_french, path_tosave)

So far, in this section, we have been working with the bios of the markers. We deleted all rows that were from brands that were not french. The final output of this section includes two dfs: one for the french brands and one for other brands (only selected columns)

In the next section, we will load the data with user bios and metadata. We will filter based on certain metrics, and also ignore all connections between followers and non french brands

## 2. Finding french brands and their followers

- In this part, we want to filter the data to only include french brands with 10 000 or more followers

In [ ]:
#Take a look at the data (Of the markers and their followers)
path = '/home/livtollanes/NewData/'
file = 'markers_followers_2023-05-19.csv'


def print_lines(path, file, start_line=0, end_line=5):
    with open(f"{path}/{file}", 'r') as f:
        for i in range(end_line):
            line = f.readline()
            if i >= start_line:
                print(line)

print_lines(path, file)


In [ ]:
#Reduce markers_followers to only include followers of french markers, as found in markers_bios_french
importlib.reload(utils)
markers_followers_french = utils.streamline_ids(markers_followers, 'marker_id', markers_bios_french, 'marker_id')


In [ ]:
# Save the filtered data
markers_followers_french.to_csv('/home/livtollanes/NewData/workdata/markers_followers_french_2023-05-19.csv', index=False)

- Now, we have data for the french markers and their followers (markers_followers_french )
- we also have a df with all french markers and their bios (markers_bios_french)

Followers per brand - only including those with 10 000 +

In [ ]:
# How many followers are there per brand? - faster to read the csv file rather than directly using the df
importlib.reload(utils)

#Create a dict of keys: brands and values: number of followers
filepath = '/home/livtollanes/NewData/workdata/markers_followers_french_2023-05-19.csv'
followers_per_brand_count = utils.create_followers_per_brand_dict(filepath)

# Filter the dictionary to only include brands with more or equal to 10,000 followers
brands_10000_followers = {brand: followers for brand, followers in followers_per_brand_count.items() if followers > 9999}

# Print the number of such brands
print(len(brands_10000_followers))

#Now, keeping only the marker_Ids in the two relevant dfs based on the brands_10000_followers
markers_bios_french_10k = utils.streamline_ids_dict(markers_bios_french, 'marker_id', brands_10000_followers)
markers_followers_french10k = utils.streamline_ids_dict(markers_followers_french, 'marker_id', brands_10000_followers)


Now, our french marker_follower data is filtered so that it only includes french brands with 10 000 followers or more

In [ ]:
#Streamline the follower_bios to only include the followers of the 10k brands
follower_bios_french10k = utils.streamline_ids(follower_bios, 'follower_id', markers_followers_french10k, 'follower_id')

In [ ]:
# Check for missing values in the 'follower_id' column of the target DataFrame
missing_values_in_target = follower_bios['follower_id'].isnull().sum()
print(f"Number of missing values in 'follower_id' column of target DataFrame: {missing_values_in_target}")

# Check for duplicate values in the 'follower_id' column of the target DataFrame
duplicate_values_in_target = follower_bios['follower_id'].duplicated().sum()
print(f"Number of duplicate values in 'follower_id' column of target DataFrame: {duplicate_values_in_target}")

Something is wrong in the bio data. after filtering to only include the ids that are in the markers_followers_french10k data, the target (bio data) becomes smaller. This must mean that some ids for which we have follower info on, do not exist in the bio data

In [ ]:
#Write the three files to csv
markers_bios_french_10k.to_csv('/home/livtollanes/NewData/workdata/step2/french_markers_bios_10k_2023-05-19.csv', index=False)
markers_followers_french10k.to_csv('/home/livtollanes/NewData/workdata/step2/french_markers_followers_10k_2023-05-19.csv', index=False)
follower_bios_french10k.to_csv('/home/livtollanes/NewData/workdata/step2/french_follower_bios_10k_2023-05-19.csv', index=False)

In [ ]:
# #count the unique ids in french_marker_bios and in filtered_markers_followers
# print(french_marker_bios['id'].nunique())
# print(markers_followers['id'].nunique())
# print(french_markers_followers['id'].nunique())

# #print the unique ids that occur in french_marker_bios but not in filtered_markers_followers
# print(set(french_marker_bios['id']) - set(french_markers_followers['id']))

# # Get the unique ids that occur in dfm_french but not in filtered_markers_followers
# missing_ids = set(french_marker_bios['id']) - set(filtered_markers_followers['id'])

# # Filter french_marker_bios to only include rows with these ids
# missing_brands = french_marker_bios[french_marker_bios['id'].isin(missing_ids)]

# # Print the missing brands
# print(missing_brands)


Obs: For three brands, existing in the french brands bio df, there are no data in the marker_follower data. Must inspect later

## 3. Filtering users

- Now that we have the french brands with enough followers, we want to remove users that don't follow enough of these 
- This is to match the procedure of He and Tsvetkova (2023)
- The purpose is to ensure we have enough information to generate SES estimates for users.

Create dictionary of [keys: follower_Id, Value: Brand_id]
Without loading the data into python

In [ ]:
#Create a dictionary of [keys:follower_id, values: brands] based on the link between french brands and their followers
importlib.reload(utils)

filepath = '/home/livtollanes/NewData/workdata/step2/french_markers_followers_10k_2023-05-19.csv'
brands_per_follower, brands_per_follower_count = utils.create_brands_per_follower_dict(filepath)

#How many unique values (brands) are there in the dictionary?

utils.inspect_dict(brands_per_follower, 5)



In [ ]:
# How many people follow less than five french brands? Remove these from the dictionary
num_brands = 5
num_items = 5
filtered_brands_per_follower = utils.inspect_and_filter_followers(brands_per_follower, num_brands, num_items, remove=True)

In [ ]:
#Filter the follower bio file to only include the followers that follow at least 5 of the french brands with more than 10 000 followers
importlib.reload(utils)
follower_bios_french10k_5 = utils.streamline_ids_dict(follower_bios_french10k, 'follower_id', filtered_brands_per_follower)

Remove all users that have sent less than 100 tweets

In [ ]:
# Filter the DataFrame to only include users with more than 100 tweets
follower_bios_french10k_5_100 = follower_bios_french10k_5[follower_bios_french10k_5['tweets'] > 99]

#print the number of rows in the filtered df
print(f"Number of rows after removing all with less than 100 tweets: {follower_bios_french10k_5_100.shape[0]}")

#Print how many rows were Removed
print(f"Number of rows removed: {follower_bios_french10k_5.shape[0] - follower_bios_french10k_5_100.shape[0]}")

In [ ]:
#  Only include users with more than 25 followers
follower_bios_french10k_5_100_25 = follower_bios_french10k_5_100[follower_bios_french10k_5_100['followers'] > 24]

# Print the number of rows in the filtered DataFrame   
print(f"Number of rows after removing all with less than 25 followers: {follower_bios_french10k_5_100_25.shape[0]}")

# Print how many rows were removed
print(f"Number of rows removed: {follower_bios_french10k_5_100.shape[0] - follower_bios_french10k_5_100_25.shape[0]}")

In [ ]:
#streamline the ids between follower_bios_french10k_5_100_25 and markers_followers_french10k
importlib.reload(utils)
markers_follower_french_10_5_100_25 = utils.streamline_ids(markers_followers_french10k,'follower_id', follower_bios_french10k_5_100_25 , 'follower_id')

In [ ]:
# Now write the filtered_df3 to a csv file
follower_bios_french10k_5_100_25.to_csv('/home/livtollanes/NewData/workdata/step2/step3/follower_bios_french10k_5_100_25.csv', index=False)
markers_follower_french_10_5_100_25.to_csv('/home/livtollanes/NewData/workdata/step2/step3/markers_follower_french_10_5_100_25.csv', index=False)

### Filterings so far
- Users that follow french brands with 10 000 or more followers
- Users that follow at least five brands
- Have sent at least 100 tweets
- Have at least 25 followers


### Filters I have not conducted
- Filters based on user location or language
- Only include users that have sent at least five tweets in the first few months of the year the data was collected (maybe not relevant for this data - the twitter bios data does not contain the tweets. Only creation date for the profile)
